In [ ]:
import polars as pl
import altair as alt
from vega_datasets import data
import pandas as pd

alt.data_transformers.enable("vegafusion")

In [ ]:
alt.themes.enable("opaque")

How many H1B petitioners are there for each year? What sectors are they in?

In [ ]:
uscis_petitioners_2021_2024 = pl.read_excel("../data/uscis_h1b_approvals_2021_2024.xlsx")
uscis_petitioners_2017_2020 = pl.read_excel("../data/uscis_h1b_approvals_2017_2020.xlsx")
uscis_petitioners = uscis_petitioners_2017_2020.vstack(uscis_petitioners_2021_2024)
uscis_petitioners

In [4]:
uscis_petitioners = uscis_petitioners.rename({"Line by line": "id",
                                              "Fiscal Year   ": "fiscal_year",
                                              "Employer (Petitioner) Name": "name",
                                              "Tax ID": "tax_id",
                                              "Industry (NAICS) Code": "naics_code",
                                              "Petitioner City": "city",
                                              "Petitioner State": "state",
                                              "Petitioner Zip Code": "zip",
                                              "Initial Approval": "initial_approval",
                                              "Initial Denial": "initial_denial",
                                              "Continuing Approval": "continuing_approval",
                                              "Continuing Denial": "continuing_denial"})

# TODO: clean naics_code column among others here

In [ ]:
# Check to see if USCIS data contains duplicate records of employers

uscis_petitioners.n_unique(subset=["fiscal_year", "tax_id"]) # there are 79244 counts of (employers, year)

petitioners_count = uscis_petitioners.group_by("fiscal_year", "tax_id").len(name="employers_count")
petitioners_count # there are indeed dupliates

In [6]:
# Construct a new df with each row representing unique employers only
petitioners_df = uscis_petitioners.unique(subset=["fiscal_year", "tax_id"])
petitioners_df = petitioners_df.with_columns(
    pl.when(pl.col("naics_code").str.contains("54")) # do with regex
    .then(pl.lit("Professional, Scientific, and Technical Services"))
    .when(pl.col("naics_code").str.contains("31-33"))
    .then(pl.lit("Manufacturing"))
    .when(pl.col("naics_code").str.contains("62"))
    .then(pl.lit("Healthcare and Social Assistance"))
    .when(pl.col("naics_code").str.contains("52"))
    .then(pl.lit("Finance and Insurance"))
    .when(pl.col("naics_code").str.contains("51"))
    .then(pl.lit("Information"))
    .when(pl.col("naics_code").str.contains("61"))
    .then(pl.lit("Educational Services"))
    .when(pl.col("naics_code").str.contains("42"))
    .then(pl.lit("Wholesale Trade"))
    .otherwise(pl.lit("Other"))
    .alias("sector")
)

In [ ]:
# Because there are many sectors, we extract the first 5 popular so as not to clutter the chart
top_naics_by_petitioners = (petitioners_df.group_by(["fiscal_year", "naics_code"])
                            .len().filter(pl.col("fiscal_year") == "2024")
                            .sort("len", descending=True))
top_naics_by_petitioners.head(10)

In [ ]:
# Number of companies filing don't change much over the years (about 10k)
# A slight but discernible increase in need for foreign talent in the tech sector.
# Industry profile of H1B petitioners remain fairly consistent
alt.Chart(petitioners_df).mark_area().encode(
    alt.X("fiscal_year", type="nominal"),
    alt.Y(aggregate="count", type="quantitative", title="employers_count"),
    alt.Color("sector", scale=alt.Scale(scheme='tableau10'))
).properties(
    title="Employers filing for H1B petitions (2017-2024)"
)

What about the number of H1B filings?

In [ ]:
# Verify 2017-2020 cap-subjected filings
uscis_filings = pl.read_excel("../data/uscis_h1b_eligible_registration_2017_2020.xlsx")
base = alt.Chart().mark_bar(color="#384d26").encode(
    alt.X("fiscal_year", type="nominal"),
    alt.Y("count", type="quantitative", title="total_cap_subjected_filings")
)

line = (alt.Chart(pl.DataFrame({"y": 85000}))
        .mark_rule(color="#EBEBEB")
        .encode(alt.Y("y", type="quantitative"))
)

alt.layer(base, line, data=uscis_filings).properties(title="H1B filings (2017-2024)")

Now diving in data at individual level. Due to [...], we examine H1B lottery data (filings that are subjected to cap) 2023

In [10]:
# mosaic chart to profile nationality of beneficiary/filer (single/multiple regis)
# choloreth heat map of US (shading for employees, spots for worksite)
# text over heatmap for denial rate for race, class type
# occupation types and compensation (scatterplot?)
# chance (master, bachelor, single filer, multiple filer?)
# compare compensation with domestic for some job groups
# track h1b selection and unemployment

In [11]:
beneficiaries = pl.read_csv("../data/bloomberg_h1b_records_2023.csv", ignore_errors=True)
beneficiaries = beneficiaries.select(["country_of_nationality", "ben_year_of_birth", "gender", "employer_name",
                      "FEIN", "state", "lottery_year", "status_type", "rec_date", "FIRST_DECISION", "ben_multi_reg_ind",
                      "first_decision_date", "BEN_CURRENT_CLASS", "REQUESTED_CLASS", "BASIS_FOR_CLASSIFICATION", "JOB_TITLE",
                      "WORKSITE_ZIP", "BEN_PFIELD_OF_STUDY", "BEN_COMP_PAID", "S3Q1",
                      "DOT_CODE", "NAICS_CODE"]).rename({"country_of_nationality": "nationality",
                                                         "ben_year_of_birth": "birthyear",
                                                         "state": "employer_state",
                                                         "FEIN": "fein",
                                                         "NAICS_CODE": "naics_code",
                                                         "lottery_year": "fiscal_year",
                                                         "FIRST_DECISION": "first_decision",
                                                         "BEN_CURRENT_CLASS": "current_class",
                                                         "REQUESTED_CLASS": "requested_class",
                                                         "BASIS_FOR_CLASSIFICATION": "filing_type",
                                                         "JOB_TITLE": "job_title",
                                                         "WORKSITE_ZIP": "worksite_zip",
                                                         "BEN_PFIELD_OF_STUDY": "field_of_study",
                                                         "DOT_CODE": "dot_code",
                                                         "BEN_COMP_PAID": "base_income",
                                                         "S3Q1": "filing_category"})

# Remove filing category "exempt" (we care about cap-subjected entries only), 
# select only new employment (lottery-participating entries), and care only about 1B1 (not HSC)
# requested_class.

# Incorrect state in data file. Modify TS to TN
beneficiaries = beneficiaries.with_columns(pl.when(pl.col("employer_state") == "TS")
                             .then(pl.lit("TN")).otherwise(pl.col("employer_state"))
                             .alias("employer_state_corrected")).drop("employer_state")

In [12]:
# Pad zip with 0
beneficiaries = beneficiaries.with_columns(pl.col("worksite_zip").cast(pl.Utf8).str.zfill(5))
zipcodes = pl.read_csv("../data/zipcodes.csv")
zipcodes = zipcodes.with_columns(pl.col("zip_code").cast(pl.Utf8).str.zfill(5))
beneficiaries_worksite = beneficiaries.join(zipcodes, how="left", left_on="worksite_zip", right_on="zip_code")

beneficiaries_worksite = beneficiaries_worksite.filter(~pl.col("worksite_zip").is_null())

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature="states")
beneficiaries_bystate = beneficiaries.unique("fein").group_by("employer_state_corrected").len(name="filings_count")

# Reference: https://stackoverflow.com/questions/66892810/using-transform-lookup-for-an-altair-choropleth-figure
pd_state_code = pd.read_csv('https://www2.census.gov/geo/docs/reference/state.txt', sep="|")
pd_state_code.columns = ['id', 'abbr', 'state', 'statens']
pd_state_code = pd_state_code[['id', 'abbr', 'state']]
pl_state_code = pl.from_pandas(pd_state_code)

beneficiaries_bystate = beneficiaries_bystate.join(pl_state_code, how="left", left_on="employer_state_corrected", right_on="abbr")

base = alt.Chart(states).mark_geoshape(fill='black', stroke='black', strokeWidth=0.5)

chart = alt.Chart(states).mark_geoshape().encode(
    color=alt.Color("filings_count:Q", scale=alt.Scale(scheme='lightorange'), title="petitioners_count")
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(beneficiaries_bystate, "id", ["filings_count"])
).properties(
    width=500,
    height=300
).project("albersUsa")

dot = alt.Chart(beneficiaries_worksite).mark_circle(color="#454722").encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    tooltip='worksite_zip:N',
    size="count()"
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.layer(base, chart, dot).properties(title="Petitioners and selected H1B workers worksite density across the US (2023)")

What about nationality profiling?

In [ ]:
filings_by_nationality = beneficiaries.group_by(pl.col("nationality")).len()
filings_by_nationality.sort("len", descending=True).head(10)
filings_by_nationality = filings_by_nationality.with_columns(pl.when(~pl.col("nationality").is_in(["IND", "CHN", "CAN", "MEX", "PHL", "KOR"]))
                                    .then(pl.lit("OTHERS"))
                                    .otherwise(pl.col("nationality"))
                                    .alias("nationality_shortlisted")).drop("nationality").drop_nulls()


alt.Chart(filings_by_nationality).mark_arc().encode(
    theta="len",
    color=alt.Color("nationality_shortlisted", scale=alt.Scale(scheme='tableau10'))
).properties(title="Filings by nationality (2023)")

What of an individual chances?

In [ ]:
beneficiaries_by_filing = beneficiaries.filter(~pl.col("ben_multi_reg_ind").is_null())

filing_chance = beneficiaries_by_filing.group_by(["ben_multi_reg_ind", "status_type", "first_decision"]).len()
filing_chance

In [ ]:
# How many multiple filers? How many single filers?
base = alt.Chart(beneficiaries_by_filing).encode(
    x='count()',
    y="ben_multi_reg_ind:N",
    text='count()'
)
base.mark_bar(color="#454722").properties(title="Multiple vs. Single filings (2023)") + base.mark_text(align='left', dx=2)

In [ ]:
# How many Bachelor's degree holders? How many Master's?
beneficiaries_by_degree = beneficiaries_by_filing.filter(pl.col("filing_category") != "E").group_by("filing_category").len()
beneficiaries_by_degree = beneficiaries_by_degree.with_columns(pl.when(pl.col("filing_category") == "B")
                                     .then(pl.lit(65000))
                                     .otherwise(pl.lit(85000))
                                     .alias("cap"))


bar = alt.Chart(beneficiaries_by_degree).mark_bar(color="#454722").encode(
    alt.X("filing_category"),
    y="len"
).properties(
    width=alt.Step(40)
)

tick = alt.Chart(beneficiaries_by_degree).mark_tick(
    color='#a91825',
    thickness=2,
    size=40*0.9,
).encode(
    x='filing_category',
    y='cap'
)

alt.layer(bar, tick).properties(title="Filings by degree (2023)")

In [18]:
# Chance to get picked from lottery
lottery_win = filing_chance.with_columns((pl.col("len") / pl.sum("len")).alias("pct")).filter(pl.col("status_type") == "ELIGIBLE")
lottery_win = lottery_win.with_columns((1 - pl.col("pct")).alias("winning_rate"))

chance_1 = alt.Chart(lottery_win).mark_bar(color="#454722").encode(
    alt.X("ben_multi_reg_ind", type="nominal"),
    alt.Y("winning_rate", type="quantitative").scale(domain=(0, 1))
)

In [19]:
# Conditional on being picked, chance to get selected
approval_rate = filing_chance.filter((pl.col("status_type") != "ELIGIBLE") & (~pl.col("first_decision").is_null()))
approval_rate = approval_rate.group_by(["ben_multi_reg_ind"], ).agg(pl.col("len").sum()).join(approval_rate, how="right", on="ben_multi_reg_ind").with_columns((pl.col("len_right") / pl.col("len")).alias("approved_rate"))
approval_rate = approval_rate.filter(pl.col("first_decision") == "Approved")

chance_2 = alt.Chart(approval_rate).mark_bar(color="#454722").encode(
    x="ben_multi_reg_ind:N",
    y="approved_rate:Q"
)

In [20]:
overall_chance = approval_rate.select(["ben_multi_reg_ind", "approved_rate"]).join(lottery_win.select(["ben_multi_reg_ind", "winning_rate"]), how="left", on="ben_multi_reg_ind").with_columns((pl.col("winning_rate") * pl.col("approved_rate")).alias("h1b_rate"))

overall = alt.Chart(overall_chance).mark_bar(color="#454722").encode(
    alt.X("ben_multi_reg_ind", type="nominal"),
    alt.Y("h1b_rate", type="quantitative").scale(domain=(0, 1))
)

In [ ]:
# Chance analysis chart
alt.hconcat(chance_1, chance_2, overall).properties(
    title="H1B lottery chance analysis (2023)"
)

In [22]:
# TODO: Denial rate (or approval?) heatmap for singular filers

Now we move on to income analysis for those who are selected in the lottery and approved for H1B visa

In [23]:
beneficiaries_income = beneficiaries.filter(~pl.col("base_income").is_null()).with_columns(pl.col("dot_code").cast(pl.Utf8).str.zfill(3))
job_code = pl.read_excel("../data/i129_job_codes.xlsx")
job_code = job_code.rename({"Occupation Category": "category", "Occupation Code": "dot_code", "Occupation Description": "occupation"})
beneficiaries_income = beneficiaries_income.join(job_code, how="left", on="dot_code")
beneficiaries_income = beneficiaries_income.with_columns((pl.col("fiscal_year") - pl.col("birthyear")).alias("age"))
beneficiaries_income = beneficiaries_income.filter((pl.col("base_income") > 10000) & (pl.col("base_income") < 500000) & (pl.col("age") > 18) & (~pl.col("category").is_null()))

In [ ]:
alt.Chart(beneficiaries_income).mark_point().encode(
    x="age:Q",
    y="base_income:Q",
    color=alt.Color("gender", scale=alt.Scale(
                    domain=["male", "female"], range=["#ba7246", "#384d26"])),
    facet=alt.Facet("category:N",
                    columns=3,
                    spacing=5,
                    header=alt.Header(labelFontSize=8))).properties(title="Compensation by Age (2023)", width=100, height=100
                    )